In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import zenhan
from jeraconv import jeraconv
j2w = jeraconv.J2W()

import subprocess
import os

In [2]:
FILE_LIST = [    
    "./rawdata/kaitou_8_gyoukaku_20210914.xlsx",
    "./rawdata/kaitou_6_kisei_20210914.xlsx",
    "./rawdata/teian_4_gyoukaku_20210914.xlsx",
    "./rawdata/teian_2_kisei_20210914.xlsx",
]
JSON_SUFFIX = "_processed" 

In [3]:
baseurl = "https://www.gyoukaku.go.jp/hotline/img/"
for filepath in FILE_LIST:
    if not os.path.exists(filepath):
        url = baseurl + os.path.basename(filepath)
        subprocess.call(['wget', '--directory-prefix', './rawdata', url], shell=True)

In [4]:
df = pd.read_excel(FILE_LIST[0], header=[0,1])
df.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df.columns.ravel()]
df = df.dropna(subset=["番号"])

C:\Users\tsasa\miniconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  


In [5]:
def helper(x):
    if x!='nan' and x is not None:
        try:
            y = str(j2w.convert('令和' + zenhan.z2h(str(x[-1])) + '年')) + '年'
        except:
            return ''
    else:
        y = ''
    return y

def conv_date(df, column):
    tmp = df[column].apply(lambda x: str(x).replace('\n','')).str.split('年', expand=True)
    tmp.rename(columns={0: 'Y', 1: 'MD'}, inplace=True)
    try:
        tmp['MD'] = tmp['MD'].apply(lambda x: zenhan.z2h(str(x)) if x is not None else '')
    except:
        print(tmp)
        pass
    #tmp['Y'] = tmp['Y'].apply(lambda x: str(j2w.convert('令和' + zenhan.z2h(str(x)) + '年')) + '年' if x != 'nan' and x is not None else '')
    tmp['Y'] = tmp['Y'].apply(lambda x: helper(x))
    tmp['YMD'] = tmp['Y'] + tmp['MD']
    return tmp['YMD'].apply(lambda x: pd.to_datetime(x, format="%Y年%m月%d日") if x != '' else '')

In [6]:
#df['所管省庁への検討要請日'] = conv_date(df, '所管省庁への検討要請日')
#df['回答取りまとめ日'] = conv_date(df, '回答取りまとめ日')

In [7]:
df['所管省庁への検討要請日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df['所管省庁への検討要請日'],'D')
df['回答取りまとめ日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df['回答取りまとめ日'],'D')

In [8]:
df.head()

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案の具体的内容,提案理由,提案主体,所管省庁,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,備考
0,1.0,2020-10-21,2020-11-11,税務署における事前予約制,予約がない納税者に対し、受け付けず、改めての来署を指導しています。自分としては、やりすぎだと...,その場で処理できる内容であっても、事前予約規制を徹底するために出直しをさせるのは、納税者にと...,個人,財務省,国税に関するご相談について、具体的に書類や事実関係を確認する必要があるものについては、納税...,－,その他,制度の現状欄に記載のとおりです。,NaN
1,2.0,2020-10-21,2020-11-11,高速道路周回割引,高速道路周回割引は各社ごとのエリア限定となっており、鉄道の周遊券のように会社をまたがってでき...,エリア別に分割したのは政府と旧道路公団です。\n顧客目線になっていない。,個人,国土交通省,高速道路の周遊割引は、高速道路会社において、地域活性化や観光振興を目的として、期間や区間を...,－,事実誤認,高速道路会社の実施する周遊割引の対象エリアについては高速道路会社が設定しているものです。いた...,NaN
2,3.0,2020-10-21,2020-11-11,所得制限について,中学生で病気を発症し難病になった子を抱えております。行政の申請案内にある福祉関係は所得制限を...,・病気になり診断を受けたら、病院でも福祉の案内、申請ができるようにする。\n・病気で福祉の申...,個人,厚生労働省,・指定難病の患者への医療費助成については、患者の負担軽減を図ると共に、治療研究を推進するため...,難病の患者に対する医療等に関する法律等,対応不可,・難病の医療費助成については、難病の患者に対する医療等に関する法律（平成26年法律第50号）...,NaN
3,4.0,2020-10-21,2020-11-11,日銀本店の機構改革,日銀本店に勤務する人員（職員、警備員）の削減。オフィススペースの有効活用が必要と思われます,先日、破損した紙幣の交換のため、生まれて初めて日銀本店（日本橋本石町）の中に入りました。先ず...,個人,財務省,日本銀行の独立性を尊重する観点から、組織規程等については、財務省は届出を受けるのみとなってお...,日本銀行法第59条,その他,制度の現状欄に記載のとおりです。,NaN
4,5.0,2020-10-21,2020-11-11,スマホによる転記を求める,手書きで転記可能な書類のスマホ撮影を可能にして欲しい,仕事柄、閲覧可能な公文書のなかから、必要な情報を取得して持ち帰ることが多いです（建築計画概要...,個人,内閣府,○国立公文書館利用等細則（特定歴史公文書等の撮影）\n 国立公文書館の利用者が、特定歴史公...,左記のとおり,その他,一般的に公文書の閲覧に係る制約のあり方については、制度を所管している主体や、実際に文書を保有...,NaN


In [9]:
print(FILE_LIST[1])
df2 = pd.read_excel(FILE_LIST[1], header=[0,1])
df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]
df2 = df2.dropna(subset=["番号"])
#df2['所管省庁への検討要請日'] = conv_date(df2, '所管省庁への検討要請日')
#df2['回答取りまとめ日'] = conv_date(df2, '回答取りまとめ日')

#df2['所管省庁への検討要請日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df2['所管省庁への検討要請日'],'D')
df2['所管省庁への検討要請日'] = pd.to_datetime(df2['所管省庁への検討要請日'])
#df2['回答取りまとめ日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df2['回答取りまとめ日'],'D')
df2['回答取りまとめ日'] = pd.to_datetime(df2['回答取りまとめ日'])

./rawdata/kaitou_6_kisei_20210914.xlsx


C:\Users\tsasa\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df2.head()

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案の具体的内容,提案理由,提案主体,所管省庁,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,ワーキング・グループにおける処理方針
0,1.0,2020-10-19,2020-11-24,\n技術士（農業部門）及び技術士補（農業部門）登録者の（農業）普及指導員への任用について\n,\n平成16年度まで普及職員の資格試験は「改良普及員」と「専門技術員」の２つが実施されていま...,\n普及指導員は高度な普及事業を担当する即戦力の技術者として任用されることから「普及指導員資...,\n個人\n,\n農林水産\n,\n普及指導員は、都道府県職員の役職であり、都道府県知事が任用するものです。このため、原則普...,\n農業改良助長法施行令（昭和27年政令第148号）第３条第２号、農業改良助長法施行令に規定...,\n現行制度下で対応可能\n,\n「農業改良助長法施行令（昭和27年政令第148号）第３条第２号」及び「農業改良助長法施行...,NaN
1,2.0,2020-10-19,2020-11-09,\n法務局の登記簿謄本と公図のデジタル化\n,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n個人\n,\n法務省\n,\n不動産登記手続及び商業・法人登記手続においては，登記事項証明書，地図，建物所在図，地図に...,\n不動産登記法第１１９条及び１２０条，商業登記法第１０条，電気通信回線による登記情報の提供...,\n対応不可\n,\n登記制度は，登記記録に記録された内容を公示することにより，国民の権利の保全や商号，会社等...,NaN
2,3.0,2020-10-19,2020-11-24,\n保険について\n,\nコロナ禍の中ご苦労様です。私茅ヶ崎市在住の者ですが、社会保険加入と国民健康保険の切り替え...,\n今後のデジタル化に伴いマイナンバーカードの利点をアナログからデジタル化活用し無駄な動きが...,\n個人\n,\n厚生労働省\n,\n 社会保険加入について、国保保険者に自動的に伝える仕組みについては、現時点では、社会保険...,\n\n,\n対応不可\n,\n制度の現状欄に記載のとおりです。\n,NaN
3,5.0,2020-10-19,2020-11-09,\n成人向け作品におけるモザイクや消しについて\n,\n成人向けの性的な描写におけるモザイク等の修正が必要とされている現状の無意味な規制を撤廃す...,\n現在成人向けコンテンツにおける性器に対して掛けられている修正は世界的に見てもその必要性が...,\n個人\n,\n警察庁\n法務省\n,\n刑法第175条は、「成人向けの性的な描写におけるモザイク等の修正が必要」という規制を規定...,\nなし\n,\n事実誤認\n,\n制度の現状欄に記載のとおり「規制」についての事実誤認があります。\nなお、捜査機関におい...,NaN
4,6.0,2020-10-19,2020-11-09,\n免許更新手続き\n,\n1 更新手続きにかかる無駄な費用のカット\n2 更新手続きのデジタル化\n,\n（１） 更新手続きにかかる無駄な費用のカット\nア 教本類のデジタル化\n 最も必要の...,\n個人\n,\n警察庁\n,\n１ 教本について \n 更新時講習の実施方法については、道路交通法施行規則第38条第11...,\n道路交通法施行規則第38条第11項第１号\n道路交通法第101条第１項、第５項及び第６項...,\n１\n検討を予定\n\n２\n対応不可\n\n３\n検討を予定\n,\n１ 教本について \n 教本を有効活用することは、安全な運転の継続に資するものであると考...,NaN


In [11]:
df3 = pd.read_excel(FILE_LIST[2], header=2)
df3.rename(columns={'所管省庁への\n検討要請日': '所管省庁への検討要請日', '提案事項名\n（タイトル）':'提案事項名', '提案主体名\n（会社名・団体名）':'提案主体名'}, inplace=True)
#df3['所管省庁への検討要請日'] = conv_date(df3, '所管省庁への検討要請日')
#df3['回答取りまとめ日'] = conv_date(df3, '回答取りまとめ日')
df3['所管省庁への検討要請日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df3['所管省庁への検討要請日'],'D')
df3['回答取りまとめ日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df3['回答取りまとめ日'],'D')

In [12]:
df3.head()

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項名,提案主体名,所管省庁,備考
0,1,2020-10-21,2020-11-11,税務署における事前予約制,個人,財務省,NaN
1,2,2020-10-21,2020-11-11,高速道路周回割引,個人,国土交通省,NaN
2,3,2020-10-21,2020-11-11,所得制限について,個人,厚生労働省,NaN
3,4,2020-10-21,2020-11-11,日銀本店の機構改革,個人,財務省,NaN
4,5,2020-10-21,2020-11-11,スマホによる転記を求める,個人,内閣府,NaN


In [13]:
import datetime

def excel_date(date1):
    temp = datetime.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return int(delta.days)

def normalize(data):
    if isinstance(data, int):
        return data
    elif isinstance(data, float):
        return data
    elif isinstance(data, str):
        Y, MD = data.replace('\n', '').split('年')
        try:
            Y = str(j2w.convert(zenhan.z2h(Y) + '年')) + '年'
        except:
            Y = str(j2w.convert('令和' + zenhan.z2h(Y) + '年')) + '年'
        MD = zenhan.z2h(str(MD))
        date = datetime.datetime.strptime(Y+MD, '%Y年%m月%d日')
        return excel_date(date)
    else:
        return data

def conv_date2(df, column):
    tmp = df[column].apply(normalize)
    return tmp

In [14]:
print(FILE_LIST[3])
df4 = pd.read_excel(FILE_LIST[3], header=3)
df4.rename(columns={'所管省庁への\u3000\u3000検討要請日': '所管省庁への検討要請日', '内閣府での\n回答取りまとめ\n日':'回答取りまとめ日', '提案事項名\n（タイトル）':'提案事項名', '提案主体名\n（会社名・団体名）':'提案主体名', 'ワーキング・グループにおける\n処理方針': 'ワーキング・グループにおける処理方針'}, inplace=True)
#df4['所管省庁への検討要請日'] = conv_date2(df4, '所管省庁への検討要請日')
#df4['所管省庁への検討要請日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df4['所管省庁への検討要請日'],'D')
df4['所管省庁への検討要請日'] = pd.to_datetime(df4['所管省庁への検討要請日'])
#df4['回答取りまとめ日'] = conv_date2(df4, '回答取りまとめ日')
#df4['回答取りまとめ日'] = pd.to_datetime('1900-01-01') + pd.to_timedelta(df4['回答取りまとめ日'],'D')
df4['回答取りまとめ日'] = pd.to_datetime(df4['回答取りまとめ日'])

./rawdata/teian_2_kisei_20210914.xlsx


In [15]:
df4.head()

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項名,提案主体名,制度の所管省庁,備考,ワーキング・グループにおける処理方針
0,1,2020-10-19,2020-11-24,技術士（農業部門）及び技術士補（農業部門）登録者の（農業）普及指導員への任用について,個人,農林水産省,NaN,NaN
1,2,2020-10-19,2020-11-09,法務局の登記簿謄本と公図のデジタル化,個人,法務省,NaN,NaN
2,3,2020-10-19,2020-11-24,保険について,個人,厚生労働省,NaN,NaN
3,4,2020-10-19,NaT,一般ゴミ規制について,個人,環境省,NaN,NaN
4,5,2020-10-19,2020-11-09,成人向け作品におけるモザイクや消しについて,個人,警察庁\n法務省,NaN,NaN


In [16]:
df4.columns

Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項名', '提案主体名', '制度の所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')

In [17]:
df3.columns, df.columns

(Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項名', '提案主体名', '所管省庁', '備考'], dtype='object'),
 Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案の具体的内容', '提案理由', '提案主体',
        '所管省庁', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
        '所管省庁の検討結果_対応の概要', '備考'],
       dtype='object'))

In [18]:
df_gyosei = pd.merge(df3, df, how='left', on='番号').drop(columns=[
    '提案事項','提案主体','所管省庁_y','備考_x'])
print(df_gyosei.columns)
def a(x):
    try:
        tmp = x['所管省庁への検討要請日_y']
        if tmp is pd.NaT:
            tmp = x['所管省庁への検討要請日_x']
    except:
        print(x)
    return tmp
df_gyosei['所管省庁への検討要請日_y'] = df_gyosei.apply(a, raw=False, axis=1)
def b(x):
    try:
        tmp = x['回答取りまとめ日_x']
        if tmp == pd.NaT:
            tmp = x['回答取りまとめ日_y']
    except:
        print(x)
    return tmp
df_gyosei['回答取りまとめ日_x'] = df_gyosei.apply(b, raw=False, axis=1)
df_gyosei = df_gyosei.drop(columns=['所管省庁への検討要請日_x', '回答取りまとめ日_y'])

df_gyosei.rename(columns={
    '所管省庁への検討要請日_y': '所管省庁への検討要請日',
    '回答取りまとめ日_x': '回答取りまとめ日',
    '提案事項名': '提案事項',
    '提案主体名': '提案主体',
    '所管省庁_x': '所管省庁',
    '備考_y': 'ワーキング・グループにおける処理方針'
}
, inplace=True)

df_gyosei['区分']='行政改革'
df_gyosei['通番']=df_gyosei['番号'].map('行-{:05d}'.format)
df_gyosei.head()

Index(['番号', '所管省庁への検討要請日_x', '回答取りまとめ日_x', '提案事項名', '提案主体名', '所管省庁_x',
       '所管省庁への検討要請日_y', '回答取りまとめ日_y', '提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状',
       '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要', '備考_y'],
      dtype='object')


,番号,回答取りまとめ日,提案事項,提案主体,所管省庁,所管省庁への検討要請日,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,ワーキング・グループにおける処理方針,区分,通番
0,1,2020-11-11,税務署における事前予約制,個人,財務省,2020-10-21,予約がない納税者に対し、受け付けず、改めての来署を指導しています。自分としては、やりすぎだと...,その場で処理できる内容であっても、事前予約規制を徹底するために出直しをさせるのは、納税者にと...,国税に関するご相談について、具体的に書類や事実関係を確認する必要があるものについては、納税...,－,その他,制度の現状欄に記載のとおりです。,NaN,行政改革,行-00001
1,2,2020-11-11,高速道路周回割引,個人,国土交通省,2020-10-21,高速道路周回割引は各社ごとのエリア限定となっており、鉄道の周遊券のように会社をまたがってでき...,エリア別に分割したのは政府と旧道路公団です。\n顧客目線になっていない。,高速道路の周遊割引は、高速道路会社において、地域活性化や観光振興を目的として、期間や区間を...,－,事実誤認,高速道路会社の実施する周遊割引の対象エリアについては高速道路会社が設定しているものです。いた...,NaN,行政改革,行-00002
2,3,2020-11-11,所得制限について,個人,厚生労働省,2020-10-21,中学生で病気を発症し難病になった子を抱えております。行政の申請案内にある福祉関係は所得制限を...,・病気になり診断を受けたら、病院でも福祉の案内、申請ができるようにする。\n・病気で福祉の申...,・指定難病の患者への医療費助成については、患者の負担軽減を図ると共に、治療研究を推進するため...,難病の患者に対する医療等に関する法律等,対応不可,・難病の医療費助成については、難病の患者に対する医療等に関する法律（平成26年法律第50号）...,NaN,行政改革,行-00003
3,4,2020-11-11,日銀本店の機構改革,個人,財務省,2020-10-21,日銀本店に勤務する人員（職員、警備員）の削減。オフィススペースの有効活用が必要と思われます,先日、破損した紙幣の交換のため、生まれて初めて日銀本店（日本橋本石町）の中に入りました。先ず...,日本銀行の独立性を尊重する観点から、組織規程等については、財務省は届出を受けるのみとなってお...,日本銀行法第59条,その他,制度の現状欄に記載のとおりです。,NaN,行政改革,行-00004
4,5,2020-11-11,スマホによる転記を求める,個人,内閣府,2020-10-21,手書きで転記可能な書類のスマホ撮影を可能にして欲しい,仕事柄、閲覧可能な公文書のなかから、必要な情報を取得して持ち帰ることが多いです（建築計画概要...,○国立公文書館利用等細則（特定歴史公文書等の撮影）\n 国立公文書館の利用者が、特定歴史公...,左記のとおり,その他,一般的に公文書の閲覧に係る制約のあり方については、制度を所管している主体や、実際に文書を保有...,NaN,行政改革,行-00005


In [19]:
df_kisei = pd.merge(df4, df2.drop(columns=['ワーキング・グループにおける処理方針']), how='left', on='番号').drop(columns=[
    '提案事項','提案主体','所管省庁', #'回答参照先'
    'ワーキング・グループにおける処理方針'])
df_kisei['所管省庁への検討要請日_y'] = df_kisei.apply(a, raw=False, axis=1)
df_kisei['回答取りまとめ日_x'] = df_kisei.apply(b, raw=False, axis=1)
df_kisei = df_kisei.drop(columns=['所管省庁への検討要請日_x', '回答取りまとめ日_y'])
df_kisei.rename(columns={
    '所管省庁への検討要請日_y': '所管省庁への検討要請日',
    '回答取りまとめ日_x': '回答取りまとめ日',
    '提案事項名': '提案事項',
    '提案主体名': '提案主体',
    '所管省庁_x': '所管省庁',
    '制度の所管省庁': '所管省庁',
    '備考_y': 'ワーキング・グループにおける処理方針'
}
, inplace=True)

df_kisei['区分']='規制改革'
df_kisei['通番']=df_kisei['番号'].map('規-{:05d}'.format)
df_kisei.head()

,番号,回答取りまとめ日,提案事項,提案主体,所管省庁,備考,所管省庁への検討要請日,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,区分,通番
0,1,2020-11-24,技術士（農業部門）及び技術士補（農業部門）登録者の（農業）普及指導員への任用について,個人,農林水産省,NaN,2020-10-19,\n平成16年度まで普及職員の資格試験は「改良普及員」と「専門技術員」の２つが実施されていま...,\n普及指導員は高度な普及事業を担当する即戦力の技術者として任用されることから「普及指導員資...,\n普及指導員は、都道府県職員の役職であり、都道府県知事が任用するものです。このため、原則普...,\n農業改良助長法施行令（昭和27年政令第148号）第３条第２号、農業改良助長法施行令に規定...,\n現行制度下で対応可能\n,\n「農業改良助長法施行令（昭和27年政令第148号）第３条第２号」及び「農業改良助長法施行...,規制改革,規-00001
1,2,2020-11-09,法務局の登記簿謄本と公図のデジタル化,個人,法務省,NaN,2020-10-19,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n不動産登記手続及び商業・法人登記手続においては，登記事項証明書，地図，建物所在図，地図に...,\n不動産登記法第１１９条及び１２０条，商業登記法第１０条，電気通信回線による登記情報の提供...,\n対応不可\n,\n登記制度は，登記記録に記録された内容を公示することにより，国民の権利の保全や商号，会社等...,規制改革,規-00002
2,3,2020-11-24,保険について,個人,厚生労働省,NaN,2020-10-19,\nコロナ禍の中ご苦労様です。私茅ヶ崎市在住の者ですが、社会保険加入と国民健康保険の切り替え...,\n今後のデジタル化に伴いマイナンバーカードの利点をアナログからデジタル化活用し無駄な動きが...,\n 社会保険加入について、国保保険者に自動的に伝える仕組みについては、現時点では、社会保険...,\n\n,\n対応不可\n,\n制度の現状欄に記載のとおりです。\n,規制改革,規-00003
3,4,NaT,一般ゴミ規制について,個人,環境省,NaN,2020-10-19,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規-00004
4,5,2020-11-09,成人向け作品におけるモザイクや消しについて,個人,警察庁\n法務省,NaN,2020-10-19,\n成人向けの性的な描写におけるモザイク等の修正が必要とされている現状の無意味な規制を撤廃す...,\n現在成人向けコンテンツにおける性器に対して掛けられている修正は世界的に見てもその必要性が...,\n刑法第175条は、「成人向けの性的な描写におけるモザイク等の修正が必要」という規制を規定...,\nなし\n,\n事実誤認\n,\n制度の現状欄に記載のとおり「規制」についての事実誤認があります。\nなお、捜査機関におい...,規制改革,規-00005


In [20]:
df_all = pd.concat([df_gyosei,df_kisei], ignore_index=True)

In [21]:
df_all.tail()

,番号,回答取りまとめ日,提案事項,提案主体,所管省庁,所管省庁への検討要請日,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,ワーキング・グループにおける処理方針,区分,通番,備考
2604,1577,2021-01-27,マイナンバーカード活用,個人,厚生労働省\n総務省,2020-12-18,障害者手帳 愛の手帳をマイナンバーカードで代替。手帳廃止。\n国民健康保険以外の健康保険もマ...,手帳等 証明書発行コスト削減。\n,番号112の回答をご参照ください,NaN,NaN,NaN,NaN,規制改革,規-01577,番号112を参照
2605,1578,2021-01-27,車検制度改革,個人,国土交通省,2020-12-18,現在、初年度登録後初回車検は3年後、2回目以降は2年おきとなっているが、これを初回車検は4年...,現在の車検制度は、戦後まもなくの、自動車が突発的かつ重大な故障が発生し得るものであった時代...,番号77の回答をご参照ください,NaN,NaN,NaN,NaN,規制改革,規-01578,番号77を参照
2606,1579,NaT,引越しの手続き、また落とし物について。,個人,内閣官房\n警察庁\n総務省\n国土交通省\n厚生労働省,2021-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規-01579,NaN
2607,1580,NaT,身障者用設備（車椅子用）ステッカーシールの販売に関する件,個人,内閣府,2021-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規-01580,NaN
2608,1581,NaT,健康保険証に生涯の診察券・カルテ及び各種検査記録データー・運転免許証・マイナンバーカード他を...,個人,内閣府\n警察庁\n厚生労働省\n総務省,2021-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規-01581,NaN


In [22]:
df_all.columns

Index(['番号', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '所管省庁への検討要請日', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', 'ワーキング・グループにおける処理方針', '区分', '通番', '備考'],
      dtype='object')

## タグ付処理

In [23]:
import MeCab
tagger = MeCab.Tagger("-Ochasen -u ./dict/local_dict.dic")

In [24]:
def get_word_bag(text):
    try:
        tag = tagger.parseToNode(text)
        words = set()
        while tag:
            feature = tag.feature.split(",")
            if feature[0] == "名詞" and len(tag.surface) >= 2:
                words.add(tag.surface)
            tag = tag.next
        return list(words)
    except:
        return []

In [25]:
all_text = df_all['提案事項'] + \
           df_all['提案の具体的内容'].replace(np.nan, '', regex=True) + \
           df_all['提案理由'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_制度の現状'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_対応の概要'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_該当法令等'].replace(np.nan, '', regex=True)
tmp = all_text.apply(get_word_bag)

In [26]:
import json

with open('./dict/tag.json', encoding='utf-8') as json_file:
    rules_json = json.load(json_file)

def get_summary_tag(words):
    tags = []
    rules = rules_json['rules']

    for rule in rules:
        if any(item in words for item in rule['def']):
            tags.append(rule['tag'])
    return '\n'.join(map(str,tags))

In [27]:
df_all['タグ'] = tmp.apply(get_summary_tag)

## データ保存

In [28]:
# 概要
df_all['UID'] = df_all.index.map('UID{:05d}'.format)
df_all.drop_duplicates(subset=['通番'], inplace=True)
df_all[['UID','番号', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考', '所管省庁への検討要請日','所管省庁の検討結果_対応の分類',
        'ワーキング・グループにおける処理方針', '区分', '通番','タグ']].to_json('./s3static/src/assets/data{}.json'.format(JSON_SUFFIX), orient='records')
# 詳細
df_detailed = df_all[['UID','提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等',
       '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要']]
df_detailed = df_detailed.set_index('UID')
df_detailed.dropna(subset=['提案の具体的内容'], inplace=True) 
#df_detailed.drop_duplicates(inplace=True)     
df_detailed.to_json('./s3static/public/data/data{}.json'.format(JSON_SUFFIX), orient='index')

In [29]:
df_all

,番号,回答取りまとめ日,提案事項,提案主体,所管省庁,所管省庁への検討要請日,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,ワーキング・グループにおける処理方針,区分,通番,備考,タグ,UID
0,1,2020-11-11,税務署における事前予約制,個人,財務省,2020-10-21,予約がない納税者に対し、受け付けず、改めての来署を指導しています。自分としては、やりすぎだと...,その場で処理できる内容であっても、事前予約規制を徹底するために出直しをさせるのは、納税者にと...,国税に関するご相談について、具体的に書類や事実関係を確認する必要があるものについては、納税...,－,その他,制度の現状欄に記載のとおりです。,NaN,行政改革,行-00001,NaN,,UID00000
1,2,2020-11-11,高速道路周回割引,個人,国土交通省,2020-10-21,高速道路周回割引は各社ごとのエリア限定となっており、鉄道の周遊券のように会社をまたがってでき...,エリア別に分割したのは政府と旧道路公団です。\n顧客目線になっていない。,高速道路の周遊割引は、高速道路会社において、地域活性化や観光振興を目的として、期間や区間を...,－,事実誤認,高速道路会社の実施する周遊割引の対象エリアについては高速道路会社が設定しているものです。いた...,NaN,行政改革,行-00002,NaN,,UID00001
2,3,2020-11-11,所得制限について,個人,厚生労働省,2020-10-21,中学生で病気を発症し難病になった子を抱えております。行政の申請案内にある福祉関係は所得制限を...,・病気になり診断を受けたら、病院でも福祉の案内、申請ができるようにする。\n・病気で福祉の申...,・指定難病の患者への医療費助成については、患者の負担軽減を図ると共に、治療研究を推進するため...,難病の患者に対する医療等に関する法律等,対応不可,・難病の医療費助成については、難病の患者に対する医療等に関する法律（平成26年法律第50号）...,NaN,行政改革,行-00003,NaN,マイナンバーカード\nデジタル\n医療,UID00002
3,4,2020-11-11,日銀本店の機構改革,個人,財務省,2020-10-21,日銀本店に勤務する人員（職員、警備員）の削減。オフィススペースの有効活用が必要と思われます,先日、破損した紙幣の交換のため、生まれて初めて日銀本店（日本橋本石町）の中に入りました。先ず...,日本銀行の独立性を尊重する観点から、組織規程等については、財務省は届出を受けるのみとなってお...,日本銀行法第59条,その他,制度の現状欄に記載のとおりです。,NaN,行政改革,行-00004,NaN,,UID00003
4,5,2020-11-11,スマホによる転記を求める,個人,内閣府,2020-10-21,手書きで転記可能な書類のスマホ撮影を可能にして欲しい,仕事柄、閲覧可能な公文書のなかから、必要な情報を取得して持ち帰ることが多いです（建築計画概要...,○国立公文書館利用等細則（特定歴史公文書等の撮影）\n 国立公文書館の利用者が、特定歴史公...,左記のとおり,その他,一般的に公文書の閲覧に係る制約のあり方については、制度を所管している主体や、実際に文書を保有...,NaN,行政改革,行-00005,NaN,デジタル,UID00004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,1577,2021-01-27,マイナンバーカード活用,個人,厚生労働省\n総務省,2020-12-18,障害者手帳 愛の手帳をマイナンバーカードで代替。手帳廃止。\n国民健康保険以外の健康保険もマ...,手帳等 証明書発行コスト削減。\n,番号112の回答をご参照ください,NaN,NaN,NaN,NaN,規制改革,規-01577,番号112を参照,マイナンバーカード\n年金\nこども\n医療,UID02604
2605,1578,2021-01-27,車検制度改革,個人,国土交通省,2020-12-18,現在、初年度登録後初回車検は3年後、2回目以降は2年おきとなっているが、これを初回車検は4年...,現在の車検制度は、戦後まもなくの、自動車が突発的かつ重大な故障が発生し得るものであった時代...,番号77の回答をご参照ください,NaN,NaN,NaN,NaN,規制改革,規-01578,番号77を参照,,UID02605
2606,1579,NaT,引越しの手続き、また落とし物について。,個人,内閣官房\n警察庁\n総務省\n国土交通省\n厚生労働省,2021-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規-01579,NaN,,UID02606
2607,1580,NaT,身障者用設備（車椅子用）ステッカーシールの販売に関する件,個人,内閣府,2021-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規-01580,NaN,,UID02607
